In [3]:
import pandas as pd
import numpy as np
import json
from emoji import core
import re

In [4]:
path = './res/'

In [5]:
train = pd.read_json(path + 'train.json', typ = 'frame')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          115071 non-null  object
 1   id            115071 non-null  int64 
 2   plylst_title  115071 non-null  object
 3   songs         115071 non-null  object
 4   like_cnt      115071 non-null  int64 
 5   updt_date     115071 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.3+ MB


### 1. train 파일 내 공란 태그 삭제

In [6]:
# 공란 태그가 있는 행 확인

train[train['tags'].apply(lambda x:  any(tag == '' for tag in x))]

,tags,id,plylst_title,songs,like_cnt,updt_date
56921,"[힙합, , 흑인음악, 힙합엘이, 힙합추천, 틱톡, 외힙, HIPHOPLE]",95032,어떻게 떴냐고? 답은 바로 TikTok!,"[253222, 270262, 301533, 136007, 238347, 13384...",7,2020-01-08 11:19:11.000


In [7]:
train['tags'] = train['tags'].apply(lambda tags: [tag.strip() for tag in tags if tag.strip() != ''])

In [8]:
# 공란 태그가 제거되었는지 확인
train[train['plylst_title']=='어떻게 떴냐고? 답은 바로 TikTok!']

,tags,id,plylst_title,songs,like_cnt,updt_date
56921,"[힙합, 흑인음악, 힙합엘이, 힙합추천, 틱톡, 외힙, HIPHOPLE]",95032,어떻게 떴냐고? 답은 바로 TikTok!,"[253222, 270262, 301533, 136007, 238347, 13384...",7,2020-01-08 11:19:11.000


### 2. train 파일 내 플레이리스트 이모티콘 제거


In [9]:
# 이모지 제거 함수
def replace_emoji(inputString):
    return core.replace_emoji(inputString, replace='')

# 특수문자 제거 함수
def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^●○.@*\"※~ㆍ!<>$♥❤♡☆★♪♬♩¶♨①②ఇ¥◈✔ ͡° ͜ʖ▶◀◎ღ✿ﾉ■『│』ᕷⓗ‘|\(\)\[\]`\'…》\”\“\’·]',' ', inputString)
  return text_rmv

In [10]:
# # 이모지 제거 함수 적용
train['plylst_title'] = train['plylst_title'].apply(replace_emoji)

# 특수문자 제거 함수 적용
train['plylst_title'] = train['plylst_title'].apply(clean_text)

In [11]:
# 확인

print(train['plylst_title'][2609]) # 이모지
print(train['plylst_title'][2636]) # 특수문자

내리는 빗속의 Pop Music
2015 서재페 준비하자  


In [12]:
train['plylst_title']

0                                   여행같은 음악
1                                   요즘 너 말야
2                     편하게  잔잔하게 들을 수 있는 곡  
3                     크리스마스 분위기에 흠뻑 취하고 싶을때
4                                  추억의 노래 ㅋ
                        ...                
115066                        METAL E SM  2
115067       빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음 
115068                 1  눈물이 앞을 가리는 나의_이야기
115069    퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP
115070                  FAVORITE POPSONG   
Name: plylst_title, Length: 115071, dtype: object

In [14]:
# 양 옆 공백 제거

train['plylst_title'] = train['plylst_title'].str.strip()
train['plylst_title'] = train['plylst_title'].str.strip('%')
train['plylst_title'] = train['plylst_title'].str.strip('&')
train['plylst_title'][50:75]

50                   그루브가 넘치는 센스있는 음악들
51                             나만 들을래요
52                        내가 듣는 중화권 노래
53                  책과 함께 편안하게 듣는 연주곡들
54               이 달의 인디 포크  2018년 06월
55                           가슴시린 사랑노래
56       DJ  Lime_유럽여행에 어울리는 팝뮤직 Vol 2
57                             사무실 공기가
58                 이별 후 당신의 마음을 울리는 노래
59           불타는 금요일 저녁 더욱 기분업 시켜주는 노래
60                         가을 카페 MUSIC
61                           취향저격 앨범아트
62                    ROCK  최대한 듣기 편하게
63                    여자가 좋아하는 남자노래방곡들
64                 졸음운전방지 댄스댄스한 플레이리스트
65                 연애세포를 깨워줄 설렘 뿜뿜한 노래
66             천상지희 시간순 명곡들 중 좋아요 백개이상
67                2011 수능시험화이팅     보컬곡
68                         퇴근길 클래식 이야기
69    EDM 서바이벌 프로그램  헤드라이너 가 남긴 3가지 논쟁
70                   Mike will made it
71                  도시의밤과 어울리는 세련된 그루브
72               고속도로를 뚫고가자• • 얼른 듣자듣자
73                      이별이란거짓말  제발가지마
74                     포근한 이불속 R&B 468
Name: plylst_title, dtype

In [15]:
train.to_json('./origin_data_modify/train_mod1.json')

In [16]:
train = pd.read_json('./origin_data_modify/train_mod1.json', typ = 'frame')

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115071 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          115071 non-null  object
 1   id            115071 non-null  int64 
 2   plylst_title  115071 non-null  object
 3   songs         115071 non-null  object
 4   like_cnt      115071 non-null  int64 
 5   updt_date     115071 non-null  object
dtypes: int64(2), object(4)
memory usage: 6.1+ MB


In [18]:
# 변경사항이 잘 반영되었는지 확인
train.iloc[[56921, 2609, 2636, 2647], :]

,tags,id,plylst_title,songs,like_cnt,updt_date
56921,"[힙합, 흑인음악, 힙합엘이, 힙합추천, 틱톡, 외힙, HIPHOPLE]",95032,어떻게 떴냐고 답은 바로 TikTok,"[253222, 270262, 301533, 136007, 238347, 13384...",7,2020-01-08 11:19:11.000
2609,"[비오는날, 까페]",40321,내리는 빗속의 Pop Music,"[28832, 243951, 177460, 660937, 476680, 503352...",3,2014-07-24 11:23:19.000
2636,"[여행, 기분전환, 산책]",265,2015 서재페 준비하자,"[348950, 196398, 165299, 197626, 108268, 65788...",18,2017-09-12 23:48:23.000
2647,"[슬픔, 이별]",77888,사랑이,"[197227, 397454, 401502, 522150, 655756, 62215...",3,2007-07-08 06:18:37.000


### 3. train 파일 내 플레이리스트 중복되는 것 제거

In [19]:
# 1. 플레이리스트 아이디(id)와 플레이리스트 명(plylst_title) 추출
plylst_title = train[['id', 'plylst_title']]

# 2. 플레이리스트 이름 별 플레이리스트 ID count 테이블 생성 : plylst_id_cnt
plylst_id_cnt = plylst_title.groupby('plylst_title').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 3. 플레이리스트 명 중복 구분 : 서로 다른 플레이리스트 아이디로 겹치는 플레이리스트 이름이 없으면 '중복 없음', 그 외 '중복 있음'
plylst_id_cnt = plylst_id_cnt.assign(
    mapping_plylst_cnt_category = pd.cut(plylst_id_cnt['mapping_plylst_cnt'], [0, 1, np.inf], labels = ['중복 없음', '중복 있음'])
)
plylst_id_cnt
# [0, 1, np.inf] : 0이상 1미만 값 = 중복없음, 1이상의 값 = 중복있음

,plylst_title,mapping_plylst_cnt,mapping_plylst_cnt_category
0,,96,중복 있음
1,비 오는 날 어울리는 몽환적인 힐링 피아노 모음,1,중복 없음
2,비오는날 듣기 좋은 연주곡 모음,1,중복 없음
3,0 일렉듣고 광질준비 O K _ElecVVIP,1,중복 없음
4,00 Electronica__EDM,1,중복 없음
...,...,...,...
109455,＊＊＊nb＊＊＊,1,중복 없음
109456,ＣＬＵＢ_Music,1,중복 없음
109457,＿Now and Here,1,중복 없음
109458,ｓｅｎｓｕｏｕｓ,1,중복 없음


In [20]:
# 4. 중복 구분 별 플레이리스트 수 count 테이블 생성 : plylst_id_cnt_division
plylst_id_cnt_division = pd.DataFrame(plylst_id_cnt.groupby('mapping_plylst_cnt_category').count())
plylst_id_cnt_division

,plylst_title,mapping_plylst_cnt
mapping_plylst_cnt_category,,
중복 없음,106718,106718
중복 있음,2742,2742


In [21]:
# 플리 제목이 중복인 플리들 확인

plylst_id_cnt_dup = plylst_id_cnt[plylst_id_cnt['mapping_plylst_cnt_category'] == '중복 있음']
plylst_id_cnt_dup.sort_values('mapping_plylst_cnt', ascending=False)

,plylst_title,mapping_plylst_cnt,mapping_plylst_cnt_category
0,,96,중복 있음
106583,혼자만의 시간이 필요할때,43,중복 있음
55025,비오는날,34,중복 있음
83759,자기전 하루한곡 발라드,31,중복 있음
24584,금요일,30,중복 있음
...,...,...,...
17510,감각적인 라이징 하우스 뮤직과 함께 신나는 파티를,2,중복 있음
57041,사랑을 노래하다,2,중복 있음
57004,사랑은 깊은 만큼 아픈 것 감성힙합,2,중복 있음
57001,사랑은,2,중복 있음


In [22]:
# train에 중복 갯수, 중복 여부 컬럼 추가

train = pd.merge(train, plylst_id_cnt, on='plylst_title')
train

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,중복 없음
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,1,중복 없음
2,"[까페, 잔잔한]",76951,편하게 잔잔하게 들을 수 있는 곡,"[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,1,중복 없음
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,1,중복 없음
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,중복 없음
...,...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E SM 2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,1,중복 없음
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,1,중복 없음
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,1,중복 없음
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,1,중복 없음


- plylst_title이 빈 문자열인 플레이리스트 삭제

In [23]:
# 특수문자, 이모지를 지우고 나니 plylst_title이 빈 문자열인 플레이리스트가 96개 생김
train[train['plylst_title'] == '']

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category
2607,"[힙합, 들어보자, 취향저격, 안들으면후회, 내가좋아하는, 유명한, 랩, 많이들어본...",129675,,"[547913, 73656, 93307, 551200, 437277, 528278,...",1,2016-11-28 17:56:37.000,96,중복 있음
2608,"[기분전환, 매장음악]",11197,,"[67655, 546793, 106871, 505036, 27469, 228206,...",1,2017-04-02 14:49:22.000,96,중복 있음
2609,[휴식할],119961,,"[268922, 196430, 22802, 157601, 384047, 461341...",0,2019-08-12 11:35:53.000,96,중복 있음
2610,"[감성, 매장, 매장노래, 편집샾, Chill]",121197,,"[340439, 623012, 32006, 331691, 213839, 38630,...",1,2019-07-22 14:16:52.000,96,중복 있음
2611,"[휴식, 힐링, 밤, 새벽]",73707,,"[348193, 425017, 653264, 593013, 288934, 9308]",39,2016-01-10 16:35:58.000,96,중복 있음
...,...,...,...,...,...,...,...,...
2698,"[휴식, 봄]",56286,,"[62, 640811, 50148, 116315, 150129, 159404, 38...",0,2019-05-07 11:19:08.000,96,중복 있음
2699,"[앙, 기모띠]",146572,,"[199390, 402547, 368015, 422235, 313223, 28418...",6,2019-12-12 21:05:38.000,96,중복 있음
2700,[팝],65277,,"[108458, 131476, 26254, 454522, 593084, 20383,...",39,2020-03-11 23:02:38.000,96,중복 있음
2701,[발라드],41347,,"[93474, 461570, 12865, 130955, 157965, 141894,...",1,2016-02-10 19:26:42.000,96,중복 있음


In [24]:
train = train.sort_values(['mapping_plylst_cnt', 'plylst_title'], ascending=False)
train = train[train['plylst_title'] != '']
train # 115071 -> 114975

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category
8648,"[집중할, 혼자, 우울한, 분위기, 드라이브, 띵곡만, 산책, 발라드, 인디, 어쿠스틱]",99891,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-13 07:07:40.000,43,중복 있음
8649,"[띵_곡, 어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 인디]",80393,혼자만의 시간이 필요할때,"[585216, 389468, 95323, 508593, 613722, 455986...",1,2020-02-25 04:32:50.000,43,중복 있음
8650,"[띵_곡, 우울하고, 어쿠스틱, 혼자, 발라드, 분위기, 드라이브, 산책, 인디, 아침]",78591,혼자만의 시간이 필요할때,"[690604, 367534, 235773, 678762, 350309, 68036...",1,2020-03-06 07:17:24.000,43,중복 있음
8651,"[집중, 띵곡의, 어쿠스틱, 혼자, 발라드, 우울, 분위기, 조용한, 산책, 인디]",124580,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-03-25 06:59:25.000,43,중복 있음
8652,"[비오는날, 어쿠스틱, 혼자, 발라드, 분위기, 드라이브, 산책, 띵곡들, 인디]",106809,혼자만의 시간이 필요할때,"[629738, 13281, 193232, 389468, 335416, 95323,...",1,2019-12-08 15:34:52.000,43,중복 있음
...,...,...,...,...,...,...,...,...
32054,[일렉],56144,00 Party 일렉트로댄스,"[256418, 183424, 431998, 654442, 354592, 24132...",7,2018-02-06 16:02:42.000,1,중복 없음
17979,[일렉],58768,00 Electronica__EDM,"[673003, 251403, 617690, 441958, 436128, 62788...",5,2020-01-05 00:24:22.000,1,중복 없음
108939,[일렉],110536,0 일렉듣고 광질준비 O K _ElecVVIP,"[349306, 93890, 651902, 46845, 362347, 330768,...",173,2012-11-29 14:34:56.000,1,중복 없음
53026,"[비오는날, 추억, 회상]",106660,비오는날 듣기 좋은 연주곡 모음,"[79329, 51922, 173875, 216115, 422584, 509605,...",6,2015-10-06 11:45:24.000,1,중복 없음


In [25]:
# mapping_plylst_cnt, plylst_title, song_cnt 내림차순 정렬

train['song_cnt'] = train['songs'].apply(lambda x : len(x))
train = train.sort_values(['mapping_plylst_cnt', 'plylst_title', 'song_cnt'], ascending=False)
train

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt
8667,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81
8654,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 띵곡만, 산책, 운전, 인디]",127180,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-03-24 06:39:20.000,43,중복 있음,78
8671,"[띵_곡, 혼자만, 어쿠스틱, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 인디]",76448,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-13 05:56:55.000,43,중복 있음,78
8676,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 띵곡들, 인디]",141114,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-22 06:38:57.000,43,중복 있음,78
8683,"[어쿠스틱, 혼자, 발라드, 분위기, 드라이브, 조용한, 산책, 운전, 띵곡들, 인디]",18201,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-06 05:20:46.000,43,중복 있음,78
...,...,...,...,...,...,...,...,...,...
32054,[일렉],56144,00 Party 일렉트로댄스,"[256418, 183424, 431998, 654442, 354592, 24132...",7,2018-02-06 16:02:42.000,1,중복 없음,20
17979,[일렉],58768,00 Electronica__EDM,"[673003, 251403, 617690, 441958, 436128, 62788...",5,2020-01-05 00:24:22.000,1,중복 없음,133
108939,[일렉],110536,0 일렉듣고 광질준비 O K _ElecVVIP,"[349306, 93890, 651902, 46845, 362347, 330768,...",173,2012-11-29 14:34:56.000,1,중복 없음,59
53026,"[비오는날, 추억, 회상]",106660,비오는날 듣기 좋은 연주곡 모음,"[79329, 51922, 173875, 216115, 422584, 509605,...",6,2015-10-06 11:45:24.000,1,중복 없음,25


In [26]:
train_dup = train[train['mapping_plylst_cnt_category'] == '중복 있음']

In [27]:
train_dup['plylst_title'].nunique() 
# 중복된 플레이리스트의 유니크 타이틀 갯수 : 2741
# 원래 2742 였으나, 플레이리스트 타이틀이 공백인거 제외해 2741됨

2741

In [28]:
train_dup.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt
8667,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81
8654,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 띵곡만, 산책, 운전, 인디]",127180,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-03-24 06:39:20.000,43,중복 있음,78
8671,"[띵_곡, 혼자만, 어쿠스틱, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 인디]",76448,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-13 05:56:55.000,43,중복 있음,78
8676,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 띵곡들, 인디]",141114,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-22 06:38:57.000,43,중복 있음,78
8683,"[어쿠스틱, 혼자, 발라드, 분위기, 드라이브, 조용한, 산책, 운전, 띵곡들, 인디]",18201,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-06 05:20:46.000,43,중복 있음,78


In [30]:
# 타이틀이 중복된 플레이리스트 중 노래수를 가장 많이 포함하고 있는 플레이리스트만 남김

train_dup_filtered = pd.DataFrame(train_dup.groupby('plylst_title', group_keys=False).apply(lambda x: x.loc[x['song_cnt'].idxmax()]))
train_dup_filtered
# 2741

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt
plylst_title,,,,,,,,,
0000,"[00, 22]",3324,0000,"[447349, 445079, 21035, 667052, 698509, 656485...",2,2020-03-15 15:32:02.000,2,중복 있음,44
0506,[팝],99124,0506,"[609766, 250542, 333768, 277247, 345512, 27726...",2,2011-05-09 15:39:06.000,2,중복 있음,52
1,[Pop],92857,1,"[349248, 478100, 45823, 432333, 417181, 603175...",587,2019-11-29 04:52:35.000,5,중복 있음,147
1 1 환상의 케미스트리 211,[Pop],123815,1 1 환상의 케미스트리 211,"[243181, 520307, 291514, 496987, 285795, 68548...",0,2019-03-27 15:27:43.000,2,중복 있음,36
1 HITS OF DECADE SPECIAL,[팝],114252,1 HITS OF DECADE SPECIAL,"[527878, 344149, 156954, 492893, 33166, 218207...",1,2013-10-12 20:53:50.000,2,중복 있음,39
...,...,...,...,...,...,...,...,...,...
힙합과 일렉 그 우울한 동행 64,[Pop],28784,힙합과 일렉 그 우울한 동행 64,"[253678, 667658, 408123, 386976, 397008, 30493...",0,2019-03-27 15:27:37.000,2,중복 있음,47
힙합매장용,"[힙합, 랩]",94274,힙합매장용,"[264902, 409504, 528180, 100733, 50425, 684947...",14,2017-09-22 23:37:41.000,2,중복 있음,38
힙합패거리 전성시대 71,[힙합],137594,힙합패거리 전성시대 71,"[61329, 324775, 63998, 444465, 53224, 506463, ...",0,2019-03-27 15:28:08.000,4,중복 있음,30


In [32]:
train_dup_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2741 entries, 0000 to ＃＃90년대발라드
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   tags                         2741 non-null   object
 1   id                           2741 non-null   int64 
 2   plylst_title                 2741 non-null   object
 3   songs                        2741 non-null   object
 4   like_cnt                     2741 non-null   int64 
 5   updt_date                    2741 non-null   object
 6   mapping_plylst_cnt           2741 non-null   int64 
 7   mapping_plylst_cnt_category  2741 non-null   object
 8   song_cnt                     2741 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 214.1+ KB


In [34]:
115071 -2742

112329

In [371]:
# train_dup['mapping_plylst_cnt'].unique 
# 43, 34, 31, 30, 27, 26, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2
train_dup.groupby('mapping_plylst_cnt').count().plylst_title

mapping_plylst_cnt
2     3706
3     1302
4      664
5      350
6      378
7      266
8      192
9      153
10     180
11      33
12      72
13      52
14      42
15      45
16     144
17      17
18      90
19      19
20      40
21     105
22      22
23      46
24     120
26      52
27      27
30      30
31      31
34      34
43      43
Name: plylst_title, dtype: int64